# SpaceX Launch Sites Locations Analysis with Folium

This notebook performs geographical analysis of SpaceX launch sites using Folium.

Key components:

* Mapping all launch sites
* Visualizing success/failure of launches for each site
* Calculating distances between launch sites and nearby points of interest

## 1. Setup

In [1]:
import folium
import pandas as pd
from folium.plugins import MarkerCluster, MousePosition
from folium.features import DivIcon
from math import sin, cos, sqrt, atan2, radians
from IPython.display import display

# Constants
DATA_URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'

## 2.  Data Loading

In [2]:
# Download and read the data
spacex_df = pd.read_csv(DATA_URL)

# Select relevant columns
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]

print("First few rows of the dataset:")
display(spacex_df.head())

First few rows of the dataset:


,Launch Site,Lat,Long,class
0,CCAFS LC-40,28.562302,-80.577356,0
1,CCAFS LC-40,28.562302,-80.577356,0
2,CCAFS LC-40,28.562302,-80.577356,0
3,CCAFS LC-40,28.562302,-80.577356,0
4,CCAFS LC-40,28.562302,-80.577356,0


## 3. GeographicalAnalysis

### TASK 1: Mark all launch sites on a map

In [3]:
# Create base map
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Add marker for NASA Johnson Space Center
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
marker = folium.Marker(
    nasa_coordinate,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
    )
)
site_map.add_child(circle)
site_map.add_child(marker)

# Add markers for each launch site
for index, site in launch_sites_df.iterrows():
    circle = folium.Circle([site['Lat'], site['Long']], radius=1000, color='#000000', fill=True).add_child(folium.Popup(site['Launch Site']))
    marker = folium.Marker(
        [site['Lat'], site['Long']],
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html=f'<div style="font-size: 12; color:#000000;"><b>{site["Launch Site"]}</b></div>',
        )
    )
    site_map.add_child(circle)
    site_map.add_child(marker)

site_map

### TASK 2: Mark the success/failed launches for each site on the map

In [4]:
# Create MarkerCluster object
marker_cluster = MarkerCluster()

# Create new column for marker colors
spacex_df['marker_color'] = spacex_df['class'].apply(lambda x: 'green' if x == 1 else 'red')

# Add markers for each launch
for index, launch in spacex_df.iterrows():
    marker = folium.Marker(
        [launch['Lat'], launch['Long']],
        icon=folium.Icon(color='white', icon_color=launch['marker_color'])
    )
    marker_cluster.add_child(marker)

site_map.add_child(marker_cluster)

site_map

### TASK 3: Calculate the distances between a launch site to its proximities

In [5]:
# Add MousePosition to map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)
site_map.add_child(mouse_position)

def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6373.0
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = R * c
    return distance

# Calculate distance to coastline
launch_site = launch_sites_df.iloc[0] # First launch site
coastline_lat, coastline_lon = 28.56367, -80.57163

distance_coastline = calculate_distance(launch_site['Lat'], launch_site['Long'], coastline_lat, coastline_lon)

distance_marker = folium.Marker(
    [coastline_lat, coastline_lon],
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html=f'<div style="font-size: 12; color:#d35400;"><b>{distance_coastline:.2f} KM</b></div>'
    )
)
site_map.add_child(distance_marker)

# Draw line between launch site and coastline
coordinates = [[launch_site['Lat'], launch_site['Long']], [coastline_lat, coastline_lon]]
line = folium.PolyLine(locations=coordinates, weight=1)
site_map.add_child(line)

site_map